<a href="https://colab.research.google.com/github/StickMonkey615/Stock-Price/blob/main/Stock_LSTM_V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import dependencies

In [23]:
!apt-get update 
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium --quiet
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import selenium
import os
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Selenium dependencies
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Hit:2 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu focal-updates InRelease
Hit:5 http://security.ubuntu.com/ubuntu focal-security InRelease
Hit:6 http://archive.ubuntu.com/ubuntu focal-backports InRelease
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:9 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Reading package lists... Done
Reading package lists... Done
Building dependency tree       
Reading state information... Done
chromium-chromedriver is already the newest version (1:85.0.4183.83-0ubuntu0.20.04.3).
0 upgraded, 0 ne

Define function to scrape share price history from web

In [24]:
def get_driver(url):
    """Return web driver"""
    chrome_options = Options()
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--start-maximized')
    chrome_options.add_argument('--start-fullscreen')
    chrome_options.add_argument('--single-process')
    #chrome_options.binary_location = r"C:/python310/lib/site-packages/chromedriver_binary/"
    serv = Service(os.getcwd()+'/chromedriver')
    #chrome_driver_binary = r"C:/python310/lib/site-packages/chromedriver_binary/"
    driver = webdriver.Chrome('chromedriver', options=chrome_options, service=serv)
    driver.get(url)
    return driver

Scrape share price data

In [27]:
#!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb && apt install ./google-chrome-stable_current_amd64.deb

# selenium attempt to scrape table elements
url = "https://finance.yahoo.com/cryptocurrencies/"
url2 = "https://uk.investing.com/equities/apple-computer-inc-historical-data"
# url = "https://finance.yahoo.com/quote/AAPL/history?p=AAPL"
driver = get_driver(url)
print(driver.title)
header = driver.find_elements(By.TAG_NAME, value='th')
txt=driver.find_element(By.XPATH, value='/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/div/div/section/div[2]/table/thead/tr').text


def get_table_rows(driver):
    """Get number of rows available on the page """
    tablerows = len(driver.find_elements(By.XPATH, value='//*[@id="scr-res-table"]/div[1]/table/tbody/tr'))
    return tablerows


print(get_table_rows(driver))


# Scrape share price data from Yahoo Finance
url = "https://finance.yahoo.com/quote/AAPL/history?p=AAPL"
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")
table = soup.find_all('table')[0]
df = pd.read_html(str(table))[0]

Crypto Real Time Prices & Latest News - Yahoo Finance


NoSuchElementException: ignored

Preprocess the data

In [ ]:
# Preprocess the data
df = df.iloc[::-1]
df = df.drop(['Open', 'High', 'Low', 'Close*', 'Volume'], axis=1)
df = df.reset_index(drop=True)
df['Adj Close**'] = df['Adj Close**'].str.replace(',', '').astype(float)
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(np.array(df['Adj Close**']).reshape(-1, 1))

Split the data into training and test sets

In [ ]:
# Split the data into training and testing sets
training_data_len = int(len(df) * 0.8)
train_data = scaled_data[0:training_data_len, :]
x_train = []
y_train = []
for i in range(60, len(train_data)):
    x_train.append(train_data[i-60:i, 0])
    y_train.append(train_data[i, 0])
x_train, y_train = np.array(x_train), np.array(y_train)
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

Build, train and test the LSTM Model

In [ ]:
# Build the LSTM model
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(x_train.shape[1], 1)))
model.add(LSTM(units=50, return_sequences=False))
model.add(Dense(units=25))
model.add(Dense(units=1))

# Train the LSTM model
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(x_train, y_train, batch_size=1, epochs=1)

# Test the LSTM model
test_data = scaled_data[training_data_len-60:, :]
x_test = []
y_test = np.array(df['Adj Close**'][training_data_len:])
for i in range(60, len(test_data)):
    x_test.append(test_data[i-60:i, 0])
x_test = np.array(x_test)
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))
predictions = model.predict(x_test)
predictions = scaler.inverse_transform(predictions)

Implement the trading algorithm

In [ ]:
# Implement trading algorithm
shares = 1000
fees = 10  # Trading fees in USD
portfolio_value = []
for i in range(len(predictions)):
    if i == 0:
        portfolio_value.append(shares * df['Adj Close**'][i])
        continue
    daily_change = predictions[i] - predictions[i-1]
    if daily_change > 0:
        shares_to_buy = int((portfolio_value[i-1] - fees) / predictions[i])
        shares += shares_to_buy
        portfolio_value.append(shares * df['Adj Close**'][i])
    elif daily_change < 0:
        shares_to_sell = int((portfolio_value[i-1] - fees) / predictions[i])
        if shares_to_sell > shares:
            shares_to_sell = shares
        shares -= shares_to_sell
        portfolio_value.append(shares * df['Adj Close**'][i])
    else:
        portfolio_value.append
